In [1]:
from coinor.grumpy import GenerateRandomMIP
import coinor.grumpy
from coinor.grumpy import BBTree
from coinor.grumpy import MOST_FRACTIONAL, FIXED_BRANCHING, PSEUDOCOST_BRANCHING
from coinor.grumpy import DEPTH_FIRST, BEST_FIRST, BEST_ESTIMATE
from coinor.grumpy.BBTree import INFINITY
import time
import numpy as np
import random

from src.cylpBranchAndBound import RELIABILITY_BRANCHING, HYBRID
from src.cylpBranchAndBound import BranchAndBound

from pulp import LpVariable, lpSum, LpProblem, LpMaximize, LpConstraint,LpBinary
from pulp import LpStatus, value

In [2]:
%%capture
numVars = 30
numCons = 15
random.seed(256)
error = 0
# branch= [MOST_FRACTIONAL, FIXED_BRANCHING, PSEUDOCOST_BRANCHING,RELIABILITY_BRANCHING,HYBRID]
branch= [RELIABILITY_BRANCHING,HYBRID]
search = [DEPTH_FIRST, BEST_FIRST, BEST_ESTIMATE]
puLB = []
cyLB = []

for i in range(10):
    CONSTRAINTS, VARIABLES, OBJ, MAT, RHS = GenerateRandomMIP(numVars=numVars , numCons=numCons,rand_seed= random.randint(1, 100))
    #print('Itr %d Testing %s + %s'%(i,bs,ss))
    var   = LpVariable.dicts("", VARIABLES, 0, 1,LpBinary)
    numCons = len(CONSTRAINTS)
    numVars = len(VARIABLES)
    prob = LpProblem("relax", LpMaximize)
    prob += lpSum([OBJ[i]*var[i] for i in VARIABLES]), "Objective"
    for j in range(numCons):
        prob += (lpSum([MAT[i][j]*var[i] for i in VARIABLES])<=RHS[j],\
                     CONSTRAINTS[j])
    prob.solve()
    LB = value(prob.objective)
    for bs in branch:
        for ss in search:
            puLB.append(LB)
            
            T2 = BBTree()
            opt2, LB2 = BranchAndBound(T2, CONSTRAINTS, VARIABLES, OBJ, MAT, RHS,branch_strategy = bs,search_strategy = ss)
            cyLB.append(LB2)
            
            if abs(LB- LB2)>1e-6:
                error += 1
                #print('%s + %s has inconsistence: %g,%g'%(bs,ss,LB,LB2))

KeyboardInterrupt: 

In [3]:
print(error)
print(puLB)
print(cyLB)

0
[278.0]
[]
